In [ ]:
%pip install datasets

In [ ]:
%pip install evaluate jiwer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from transformers import WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor
from datasets import load_from_disk

# Initialize Whisper feature extractor and tokenizer
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="English", task="transcribe")

# Create a processor (optional, if you'd like to use a processor that combines both)
processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="English", task="transcribe")



def prepare_dataset(batch, feature_extractor, tokenizer):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch


# Load your dataset (make sure it's correctly loaded)
dataset_path = r"/content/drive/MyDrive/500data"
dataset_dict = load_from_disk(dataset_path)

print("dataset_dict: ")
print(dataset_dict)
print(type(dataset_dict))

# Manually iterate through the dataset and apply the prepare_dataset function
processed_data = []

# Iterate through the dataset (assuming it's split into "train" or other splits, like in the case of HuggingFace datasets)
for split_name, split_data in dataset_dict.items():
    print(f"Processing split: {split_name}")

    # Prepare each batch in the split
    processed_split_data = []
    for batch in split_data:
        processed_batch = prepare_dataset(batch, feature_extractor, tokenizer)
        processed_split_data.append(processed_batch)

    # Add the processed split to the final dataset
    processed_data.append((split_name, processed_split_data))

# Optionally: Combine processed data into a dataset-like object, depending on your use case
# If you want to convert processed data back to a DatasetDict, you can do so like this:
from datasets import DatasetDict

final_dataset_dict = DatasetDict({
    split_name: processed_split_data for split_name, processed_split_data in processed_data
})



dataset_dict: 
DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 400
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 100
    })
})
<class 'datasets.dataset_dict.DatasetDict'>
Processing split: train
Processing split: test


In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

In [ ]:
model.generation_config.language = "English"
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

In [ ]:
import evaluate

metric = evaluate.load("wer")

In [ ]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./Medical-Speech-Transcription-Whisper-Small-Fine-Tuned",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,
    learning_rate=1e-06,
    warmup_steps=200,
    num_train_epochs=8,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="epoch",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=100,
    save_strategy="epoch",
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=final_dataset_dict["train"],
    eval_dataset=final_dataset_dict["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-74-883ac3c90426>:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
processor.save_pretrained(training_args.output_dir)

[]

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Wer
1,0.006800,0.490443,11.481481
2,0.006100,0.489725,11.333333
3,0.005200,0.488747,11.333333
4,0.004400,0.487829,11.333333
5,0.003600,0.487630,11.259259
6,0.003000,0.487256,11.111111
7,0.002600,0.487303,11.185185
8,0.002300,0.487473,11.185185
9,0.002100,0.487443,11.185185
10,0.001800,0.487607,11.185185


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

TrainOutput(global_step=375, training_loss=0.003063366860151291, metrics={'train_runtime': 2030.4938, 'train_samples_per_second': 2.955, 'train_steps_per_second': 0.185, 'total_flos': 1.73151240192e+18, 'train_loss': 0.003063366860151291, 'epoch': 15.0})

In [ ]:
kwargs = {
    "dataset": "500 samples of doctor-patient conversations",
    "dataset_args": "config: English, split: test",
    "language": "en",
    "model_name": "Medical-Speech-Transcription-Whisper-Small-Fine-Tuned",
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
}

In [ ]:
trainer.push_to_hub(**kwargs)

events.out.tfevents.1738415471.589b682cc644.9339.5:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Bhaveen/500medimix/commit/3577a06869a3c6f8e733d0c808473f5d59cac4d2', commit_message='End of training', commit_description='', oid='3577a06869a3c6f8e733d0c808473f5d59cac4d2', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Bhaveen/500medimix', endpoint='https://huggingface.co', repo_type='model', repo_id='Bhaveen/500medimix'), pr_revision=None, pr_num=None)